In [17]:
import torch
import imageio
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import os
import time
import sklearn.mixture as mixture
from unit_conversion import convert_wind

In [18]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]
active_source_idx = 3
# x_max = 1
# y_max = 1
# z_max = 1
tfinal = 1.
source_location = source_points

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
df_wind_ch4['x'], df_wind_ch4['y'] = convert_wind(ws,wd)
time_dict = dict(zip(df_wind_ch4.index,zip(df_wind_ch4.x,df_wind_ch4.y)))


ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [19]:
source_points


array([[61.84826691, 40.32822479,  4.5       ],
       [99.10094831, 54.69940709,  2.        ],
       [99.89962676, 24.72759871,  2.        ],
       [23.54499552, 57.03946784,  2.        ],
       [25.09781584, 22.62636785,  2.        ]])

In [20]:
sigma=.2
model = PINN([20,20,20])
source_vals = np.array([1 if i ==active_source_idx else 0 for i in range(len(source_location))])
print(source_vals)
model.set_location(source_location,[tfinal,x_max,y_max,z_max],source_values=source_vals,sigma=sigma)


[0 0 0 1 0]


c:\Users\m10936927\Documents\methane_project\methane_pinn_dev\methane_pinn_dev-main\Net.py:12: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  torch.nn.init.xavier_uniform(m.weight)


In [21]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-4)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [22]:
n= int(5e2)
# n=1
icn = int(1e2)
sn = int(1e2)
best_loss = np.inf
max_epochs = int(2e4)
print_freq = 100

sampling_freq = 10 # how often to resample collocation and source points

for epoch in range(max_epochs):

    start_time = time.time()

    if epoch % sampling_freq == 0:

        source_collocation_points = model.source_points(sn,sigma*2) 
        ic_col = torch.cat([torch.zeros(icn,1), torch.rand(icn,1)*x_max, torch.rand(icn,1)*y_max, torch.rand(icn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max*2 - x_max*.5, torch.rand(n,1)*y_max*2- y_max*.5, torch.rand(n,1)*z_max*2 - z_max*.5], dim=1)
        # collocation_points = torch.cat([collocation_points,ic_col,source_collocation_points])
        collocation_points = torch.cat([collocation_points,source_collocation_points,ic_col])
        collocation_points.requires_grad_(True)
        # t = np.floor(collocation_points[:,0:1].detach().numpy().flatten())
        # uv = torch.tensor([time_dict[t[i]] for i in range(len(t))])
        uv = torch.ones(len(collocation_points),2)*10#wind tensor
        uv[:,1:]*= -1
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.compute_pde_loss(collocation_points,uv) # PDE residual loss

    loss = loss_1

    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    # for p in model.net.parameters():
    #     grad_norm += p.grad.data.norm(2).item()**2
    # grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))



epoch: 0, loss: 3.026e+03, grad_norm: 0.000e+00, pde_res: 3.026e+03, time: 1.772e-02
epoch: 100, loss: 1.746e+03, grad_norm: 0.000e+00, pde_res: 1.746e+03, time: 1.837e-03
epoch: 200, loss: 8.360e+02, grad_norm: 0.000e+00, pde_res: 8.360e+02, time: 1.551e-02
epoch: 300, loss: 3.635e+02, grad_norm: 0.000e+00, pde_res: 3.635e+02, time: 0.000e+00
epoch: 400, loss: 1.522e+02, grad_norm: 0.000e+00, pde_res: 1.522e+02, time: 1.636e-02
epoch: 500, loss: 7.104e+01, grad_norm: 0.000e+00, pde_res: 7.104e+01, time: 0.000e+00
epoch: 600, loss: 3.497e+01, grad_norm: 0.000e+00, pde_res: 3.497e+01, time: 1.653e-02
epoch: 700, loss: 2.074e+01, grad_norm: 0.000e+00, pde_res: 2.074e+01, time: 8.733e-03
epoch: 800, loss: 1.262e+01, grad_norm: 0.000e+00, pde_res: 1.262e+01, time: 0.000e+00
epoch: 900, loss: 8.641e+00, grad_norm: 0.000e+00, pde_res: 8.641e+00, time: 1.664e-02
epoch: 1000, loss: 5.660e+00, grad_norm: 0.000e+00, pde_res: 5.660e+00, time: 1.566e-02
epoch: 1100, loss: 5.238e+00, grad_norm: 0.0

KeyboardInterrupt: 

In [ ]:
print(x_max)

124.0


In [ ]:
model = torch.load('best_mod.m')

In [23]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = source_location[active_source_idx,2]
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, z_max, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1



In [24]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
source_loc = model.source_locs
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=True).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=.5)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[active_source_idx,0], model.source_locs[active_source_idx,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'output_gifs/test_site_example{int(time.time())}.gif', images)  # Save the images as a GIF



C:\Users\m10936927\AppData\Local\Temp\ipykernel_1736\4018537943.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [ ]:
print(source_points)

[[61.84826691 40.32822479  4.5       ]
 [99.10094831 54.69940709  2.        ]
 [99.89962676 24.72759871  2.        ]
 [23.54499552 57.03946784  2.        ]
 [25.09781584 22.62636785  2.        ]]


In [ ]:
print(time.datetime())

AttributeError: module 'time' has no attribute 'datetime'